In [19]:
import opensim as osim
import numpy as np
import hyperobjects as ho
import common
import clifford as cf
from clifford import g3c
import clifford.tools.g3c as g3c_tools
from enum import Enum


G3c, blades_g3c, stuff = g3c.layout, g3c.blades, g3c.stuff

e1, e2, e3 = g3c.e1, g3c.e2, g3c.e3
e12, e13, e23 = g3c.e12, g3c.e13, g3c.e23
e31 = -e13
Px, Py, Pz = g3c.e23, -g3c.e13, g3c.e12
up, down, homo = g3c.up, g3c.down, g3c.homo
no, ni = g3c.eo, g3c.einf
E0 = g3c.E0
I_base = g3c.I_base

def vec_to_numpy(p):
    return np.array([p | e1, p | e2, p | e3]).astype(float)

def osim_rotation_to_numpy(R: osim.Rotation):
    M = np.empty((3,3))
    for i in range(3):
        for j in range(3):
            M[i, j] = R.get(i,j)
    return M


def to_vector(v):
    if isinstance(v, osim.Vec3):
        v = v.to_numpy()
    return v[0]*e1 + v[1]*e2 + v[2]*e3

def to_point(p):
    return up(to_vector(p))

class MultiVectorType(Enum):
    NotBlade = -1
    Vector = 0
    EuclideanPoint = 1
    ConformalPoint = 2
    PointPair = 3
    Circle = 4
    Line = 5
    Sphere = 6
    Plane = 7

def blade_type(mv):
    if not isinstance(mv, cf.MultiVector):
        return MultiVectorType.NotBlade
    
    obj = g3c_tools.interpret_multivector_as_object(mv)
    if obj == -1:
        return MultiVectorType.NotBlade
    elif obj == 0:
        return MultiVectorType.Vector
    elif obj == 1:
        return MultiVectorType.EuclideanPoint
    elif obj == 2:
        return MultiVectorType.ConformalPoint
    elif obj == 3:
        return MultiVectorType.PointPair
    elif obj == 4:
        return MultiVectorType.Circle
    elif obj == 5:
        return MultiVectorType.Line
    elif obj == 6:
        return MultiVectorType.Sphere
    elif obj == 7:
        return MultiVectorType.Plane
    

def to_array(mv):
    b_type = blade_type(mv)
    if b_type == MultiVectorType.Vector:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.EuclideanPoint:
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.ConformalPoint:
        mv = down(mv)
        return np.array([mv | e1, mv | e2, mv | e3]).astype(float)
    elif b_type == MultiVectorType.Line:
        # return position then direction
        P, D = g3c_tools.line_to_point_and_direction(mv)
        if P == 0:
            return [np.array([0,0,0]), to_array(D)]
        return [to_array(P), to_array(D)]
    elif b_type == MultiVectorType.Plane:
        # return orthogonal proj of origin onto it THEN normal
        location = g3c_tools.get_nearest_plane_point(mv)
        normal = g3c_tools.get_plane_normal(mv)
        if location == 0:
            return [np.array([0,0,0]), to_array(normal)]
        return [to_array(location), to_array(normal)]
    else:
        raise TypeError(f"Can't convert {b_type} as numpy array.")


In [131]:
p = g3c_tools.random_conformal_point()
q = g3c_tools.random_conformal_point()
R = g3c_tools.random_rotation_translation_rotor()

D = no^(q-p)^ni
M = down(p) ^ (q-p) ^ ni
L = p^q^ni
print(L)
print(D + M)
print(M)

M2 = R * M * ~R
D2 = R*D*~R
print(M2)
# print(D2)
print(D2 + M2)
print('j', R * L * ~R)

L2 = R * L * ~R
_, D2 = g3c_tools.line_to_point_and_direction(L2)
D2 = no^D2^ni
# print(D2)
print("h", L2 - D2)
print("h", M2)

print(down(R * p * ~R) ^ (R * (q - p) * ~R) ^ ni)
print((down(R * p * ~R) ^ (R * (q - p) * ~R) ^ ni))

-(204.19852^e124) - (204.19852^e125) - (11.71649^e134) - (11.71649^e135) + (12.61597^e145) + (74.70903^e234) + (74.70903^e235) - (26.10094^e245) + (3.11812^e345)
-(204.19852^e124) - (204.19852^e125) - (11.71649^e134) - (11.71649^e135) + (12.61597^e145) + (74.70903^e234) + (74.70903^e235) - (26.10094^e245) + (3.11812^e345)
-(204.19852^e124) - (204.19852^e125) - (11.71649^e134) - (11.71649^e135) + (74.70903^e234) + (74.70903^e235)
(53.09634^e124) + (53.09634^e125) + (210.07079^e134) + (210.07079^e135) - (21.60546^e234) - (21.60546^e235)
(350.89212^e124) + (350.89212^e125) - (41.23979^e134) - (41.23979^e135) + (23.29866^e145) - (265.83585^e234) - (265.83585^e235) + (2.00558^e245) + (17.41535^e345)
j (350.89212^e124) + (350.89212^e125) - (41.23979^e134) - (41.23979^e135) + (23.29866^e145) - (265.83585^e234) - (265.83585^e235) + (2.00558^e245) + (17.41535^e345)
h (350.89212^e124) + (350.89212^e125) - (41.23979^e134) - (41.23979^e135) - (265.83585^e234) - (265.83585^e235)
h (53.09634^e124) +

In [132]:
print(down(p) ^ (q-p))

-(204.19852^e12) - (11.71649^e13) + (1001.06326^e14) + (1001.06326^e15) + (74.70903^e23) + (2177.57579^e24) + (2177.57579^e25) + (491.19856^e34) + (491.19856^e35)


In [61]:
print(v2)
print(v.dual())
print(blade_type(v2))
print(blade_type(v.dual()))

v3 = v + (g3c_tools.random_euc_mv() | v)*ni
print(blade_type(v3))
print(blade_type(v3.dual()))
print(v3)

(1^e12) - (2^e13) + (1^e23)
-(1^e1245) + (2^e1345) - (1^e2345)
MultiVectorType.PointPair
MultiVectorType.Sphere
MultiVectorType.Vector
MultiVectorType.Plane
(1.0^e1) + (2.0^e2) + (1.0^e3) - (22.4886^e4) - (22.4886^e5)


In [72]:
W = (e1^e2^e3)
W | v
print(e2 | v)
print((-e2 ^ v.dual()).dual())
print((-(e1^e2) | v.dual()).dual())
print((e1^e2^e3) * v)
print((e1^e2^e3) * v2)
print((e1^e2^e3) * v.dual())
print(blade_type(((e1 + e2 + e3) | v.dual())))

2
2
(1^e123)
(1^e12) - (2^e13) + (1^e23)
-(1^e1) - (2^e2) - (1^e3)
(1^e145) + (2^e245) + (1^e345)
MultiVectorType.Line


In [15]:
# A = e1 + e2 + e3
# print(A.dual())
# blade_type(A.dual())
# I = no^e1^e2^e3^ni
# print(I)
# print((A / ~I))

RI1 = g3c_tools.random_rotation_translation_rotor()
RI2 = g3c_tools.random_rotation_translation_rotor()
O = g3c_tools.random_conformal_point()
I = g3c_tools.random_conformal_point()
I1 = g3c_tools.apply_rotor(I, RI1)
I2 = g3c_tools.apply_rotor(I, RI2)
L1 = O ^ I1 ^ ni
L2 = O ^ I2 ^ ni

print(blade_type(L1))

d1 = no ^ (I1 - O).normal() ^ ni
m1 = down(O) ^ down(I1) ^ ni

d2 = no ^ (I2 - O).normal() ^ ni
m2 = down(O) ^ down(I2) ^ ni

t1 = g3c_tools.apply_rotor(d1, ~RI1)
t2 = g3c_tools.apply_rotor(d2, ~RI2)

P1 = no ^ g3c_tools.apply_rotor(down(O), ~RI1) ^ g3c_tools.apply_rotor((I1 - O).normal(), ~RI1) ^ ni
P2 = no ^ g3c_tools.apply_rotor(down(O), ~RI2) ^ g3c_tools.apply_rotor((I2 - O).normal(), ~RI2) ^ ni
print(P1)

print(blade_type(d1 / d2))
M = (I1 - O).normal() / (I2 - O).normal()
LA = g3c_tools.apply_rotor(no^(I1 - O)^ni, ~RI1)
LB = g3c_tools.apply_rotor(no^(I2 - O)^ni, ~RI2)
# print(blade_type(LA))
# print(blade_type(LA))
p =  ~RI1 * no * RI1
n =  ~RI1 * (I1 - O).normal() * RI1
q =  ~RI2 * no * RI2
m =  ~RI2 * (I2 - O).normal() * RI2


MultiVectorType.Line
-(4.88838^e1245) - (4.13645^e1345) + (4.54096^e2345)
MultiVectorType.NotBlade


In [17]:
# print((m*n) + ((n << (down(q) ^ m)) - (m << (down(p) ^ n)) - (down(p) - down(q))^m^n^ni))
# print((q ^ m ^ ni) * (ni ^ p ^ n))
# print( (-m*n) + ((q^m)*ni*n) - (m*ni*(p^n)))
# print( (m*n) + ((q^m^ni)*n) - (m*(ni^p^n)))
# print( (m*n) + ((- ((q^m) | n) - (m | (p ^ n))) ^ ni) + (q^m^ni^n) - (m^ni^p^n))
# print((m*n) + (((n << (q^m)) - (m << (p ^ n))) ^ ni) + ((q-p)^m^ni^n))

rotor = (q ^ m ^ ni) * (ni ^ p ^ n)
theta = 2 * np.arccos(float(rotor(0)))
bivec = rotor(2).normal()
print(bivec)
print(rotor)
print(np.exp(theta*0.5*bivec))
print(blade_type(bivec.dual()))
print(I1 ^ bivec.dual())
print(I2 ^ bivec)
print(O | bivec)
print(down(O) ^ bivec.dual())



-(0.91271^e12) + (0.38308^e13) + (4.39293^e14) + (4.39293^e15) - (0.14219^e23) - (0.37202^e24) - (0.37202^e25) + (2.81656^e34) + (2.81656^e35)
0.37355 - (0.84663^e12) + (0.35535^e13) + (4.07492^e14) + (4.07492^e15) - (0.1319^e23) - (0.34509^e24) - (0.34509^e25) + (2.61266^e34) + (2.61266^e35) - (7.03193^e1234) - (7.03193^e1235)
0.37355 - (0.84663^e12) + (0.35535^e13) + (3.86489^e14) + (3.86489^e15) - (0.1319^e23) - (0.91093^e24) - (0.91093^e25) + (1.26453^e34) + (1.26453^e35) - (3.36408^e1234) - (3.36408^e1235)
MultiVectorType.NotBlade
(164.94404^e1234) + (164.94404^e1235) - (12.29536^e1245) - (30.89503^e1345) - (25.78184^e2345)
-(2.96112^e123) - (241.03041^e124) - (241.94312^e125) + (115.17492^e134) + (115.558^e135) + (4.39293^e145) + (18.47497^e234) + (18.33278^e235) - (0.37202^e245) + (2.81656^e345)
(4.69876^e1) - (8.86161^e2) + (6.33213^e3) + (33.20875^e4) + (33.20875^e5)
(33.20875^e1234) + (33.20875^e1235) - (3.51557^e1245) - (8.48959^e1345) - (0.30583^e2345)


In [147]:
m*n

-0.82212 - (0.1469^e12) - (0.44443^e13) + (2.23633^e14) + (2.23633^e15) - (0.32408^e23) - (6.30806^e24) - (6.30806^e25) - (24.01725^e34) - (24.01725^e35)

In [ ]:
m*n

-0.82212 - (0.1469^e12) - (0.44443^e13) + (2.23633^e14) + (2.23633^e15) - (0.32408^e23) - (6.30806^e24) - (6.30806^e25) - (24.01725^e34) - (24.01725^e35)

In [15]:
blade_type(LB)
LB

(98.3089^e124) + (98.3089^e125) + (8.38509^e134) + (8.38509^e135) + (0.78046^e145) + (283.19321^e234) + (283.19321^e235) - (17.49777^e245) - (3.74066^e345)

In [47]:
blade_type(g3c_tools.meet(LA, LB))
t = g3c_tools.meet(LA, LB)
t = t.clean(1e-8)
t
print(O ^ t)
print(down(O) ^ t)
print(down(I) ^ t)
print(t)
print(down(O) | t)
print(down(I) | t)
print(down(I1) | t)
print(I2 | t)

(36507.31383^e14) + (36507.31383^e15) + (20005.40143^e24) + (20005.40143^e25) + (135959.16515^e34) + (135959.16515^e35) + (11338.98738^e45)
(36507.31383^e14) + (36507.31383^e15) + (20005.40143^e24) + (20005.40143^e25) + (135959.16515^e34) + (135959.16515^e35)
(139184.45651^e14) + (139184.45651^e15) + (92828.16446^e24) + (92828.16446^e25) - (230441.15445^e34) - (230441.15445^e35)
-(11338.98738^e4) - (11338.98738^e5)
0
0
0
11338.98738 - (0.0^e12) - (0.0^e14) - (0.0^e15) + (0.0^e23) - (0.0^e24) - (0.0^e25) + (0.0^e34) + (0.0^e35) + (0.0^e1234) + (0.0^e1235)


In [37]:
theta = 2 * np.arccos(float(Prot(0)))
lm = np.exp((theta / 2) * (LA * LB)(2).normal())
lm

0.24742 + (0.95275^e12) - (0.1695^e13) + (2.29796^e14) + (2.29796^e15) + (0.04826^e23) + (0.43314^e24) + (0.43314^e25) - (0.25016^e34) - (0.25016^e35) - (0.2184^e1234) - (0.2184^e1235)

In [302]:

model = osim.Model()

ground = model.getGround()
bodyA = osim.Body("bodyA", 1.0, osim.Vec3(0), osim.Inertia(0))
bodyB = osim.Body("bodyB", 2.0, osim.Vec3(0), osim.Inertia(0))
bodyC = osim.Body("bodyC", 1.0, osim.Vec3(0), osim.Inertia(0))
bodyC = osim.Body("bodyC", 1.0, osim.Vec3(0), osim.Inertia(0))

joint0 = osim.WeldJoint("joint0", ground, osim.Vec3(-1,1,2), osim.Vec3(np.pi/4, np.pi / 3, -np.pi/6), bodyA, osim.Vec3(-1,1,2), osim.Vec3(np.pi/4, np.pi / 3, -np.pi/6))
joint1 = osim.FreeJoint("joint1", bodyA, osim.Vec3(-1,0,0), osim.Vec3(0, np.pi / 3, -np.pi/6), bodyB, osim.Vec3(1,0,0), osim.Vec3(0))
joint2 = osim.WeldJoint("joint2", bodyB, osim.Vec3(-0.5,0,0), osim.Vec3(0), bodyC, osim.Vec3(0.1,0,0), osim.Vec3(0))
# joint2 = osim.PinJoint("joint2", bodyB, osim.Vec3(-0.5,0,0), osim.Vec3(0), bodyC, osim.Vec3(0.1,0,0), osim.Vec3(0), osim.Vec3(1,0.5,0.3))


muscle = osim.Millard2012EquilibriumMuscle("muscle",  # Muscle name
                                           200.0,  # Max isometric force
                                           0.6,  # Optimal fibre length
                                           0.55,  # Tendon slack length
                                           0.0)  # Pennation angle
origin = osim.Vec3(0.9, 0.8, 0.8)
muscle.addNewPathPoint("origin", bodyA, origin)

insertion = osim.Vec3(0.7, 0.7, 0.1)
muscle.addNewPathPoint("insertion", bodyC, insertion)

model.addBody(bodyA)
model.addBody(bodyB)
model.addBody(bodyC)
model.addJoint(joint0)
model.addJoint(joint1)
model.addJoint(joint2)
model.addForce(muscle)

state = model.initSystem()

In [307]:
# POSTURE 1
for c in model.getCoordinateSet():
    c.setValue(state, np.random.uniform(-np.pi, np.pi))
    # c.setValue(state, 0)
model.equilibrateMuscles(state)

J = common.station_jacobian(model, state, osim.Vec3(1,5,3), bodyC)
print(J.round(2))

Jw = common.frame_jacobian(model, state, bodyC)
print(Jw.round(2))

J = np.vstack([J, Jw])
print(J.round(2))

[[-1.54  0.9   0.51  0.43  0.25  0.87]
 [ 3.5   2.02 -4.12 -0.5   0.87  0.  ]
 [-3.22  4.56 -0.89 -0.75 -0.43  0.5 ]]
[[ 0.43  0.25  0.87  0.    0.    0.  ]
 [-0.5   0.87  0.    0.    0.    0.  ]
 [-0.75 -0.43  0.5   0.    0.    0.  ]]
[[-1.54  0.9   0.51  0.43  0.25  0.87]
 [ 3.5   2.02 -4.12 -0.5   0.87  0.  ]
 [-3.22  4.56 -0.89 -0.75 -0.43  0.5 ]
 [ 0.43  0.25  0.87  0.    0.    0.  ]
 [-0.5   0.87  0.    0.    0.    0.  ]
 [-0.75 -0.43  0.5   0.    0.    0.  ]]


In [308]:
for c in model.getCoordinateSet():
    print(c.getName())

joint1_coord_0
joint1_coord_1
joint1_coord_2
joint1_coord_3
joint1_coord_4
joint1_coord_5


In [401]:

lever_arms = np.array([muscle.computeMomentArm(state, c) for c in model.getCoordinateSet()])
print(lever_arms)

# j = bodyC
O_in_ground = bodyA.findStationLocationInGround(state, origin).to_numpy()
I_in_ground = bodyC.findStationLocationInGround(state, insertion).to_numpy()

J1_frame = osim.PhysicalOffsetFrame.safeDownCast(joint1.getParentFrame())

ground: osim.Ground = model.getGround()

O_in_joint = bodyA.findStationLocationInAnotherFrame(state, origin, J1_frame).to_numpy()
I_in_joint = bodyC.findStationLocationInAnotherFrame(state, insertion, J1_frame).to_numpy()



# t1 = np.random.uniform(600,1200) 
# W1 = np.vstack([ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, i]), J1_frame).to_numpy() for i in [0,1,2]])
# g1 = t1 * W1 @ (np.cross(I_in_joint, O_in_joint) / np.linalg.norm(u_in_joint))

# # Rotations
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 0]), J1_frame).to_numpy().dot(np.cross(I_in_joint, O_in_joint) / np.linalg.norm(u_in_joint)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 1]), J1_frame).to_numpy().dot(np.cross(I_in_joint, O_in_joint) / np.linalg.norm(u_in_joint)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 2]), J1_frame).to_numpy().dot(np.cross(I_in_joint, O_in_joint) / np.linalg.norm(u_in_joint)))
# print()

# # Translations
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 3]), J1_frame).to_numpy().dot(- u_in_joint / np.linalg.norm(u_in_joint)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 4]), J1_frame).to_numpy().dot(- u_in_joint / np.linalg.norm(u_in_joint)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 5]), J1_frame).to_numpy().dot(- u_in_joint / np.linalg.norm(u_in_joint)))

print("M =",ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[3:, 0]), J1_frame).to_numpy().dot(np.cross(I_in_joint, O_in_joint) / np.linalg.norm(I_in_joint - O_in_joint)))

O_in_joint_cf = to_point(O_in_joint)
I_in_joint_cf = to_point(I_in_joint)
L = (O_in_joint_cf ^ I_in_joint_cf ^ ni)

k = (no | L) / L
print(k)
# print(J1_frame.expressVectorInGround(state, osim.Vec3(*to_array(k))))
print(down(k))
L =L.normal()

print(L)
print()
print(L(no^e1^ni) | ~E0)
print(L(no^e2^ni) | ~E0)
print(L(no^e3^ni) | ~E0)

[ 0.58737822  0.11656302 -0.84035879  0.49415577  0.74464362  0.44868246]
M = -1.1545776820718348
-(0.88976^e1) - (0.25223^e2) + (1.39853^e3) + (2.31118^e4) + (3.31118^e5)
-(0.88976^e1) - (0.25223^e2) + (1.39853^e3)
(0.53791^e124) + (0.53791^e125) + (1.09031^e134) + (1.09031^e135) - (0.49416^e145) + (1.15458^e234) + (1.15458^e235) - (0.74464^e245) - (0.44868^e345)

(0.49416^e1)
(0.74464^e2)
(0.44868^e3)


In [402]:
# // Get the transform of the child frame in the ground frame
transform: osim.Transform = J1_frame.getTransformInGround(state)
# // Extract the rotation matrix
rotationMatrix: osim.Rotation = transform.R()

r = np.empty((3,3))
for i in [0,1,2]:
    for j in [0,1,2]:
        r[i,j] = rotationMatrix.toMat33().get(i,j)

r

array([[ 4.33012702e-01,  2.50000000e-01,  8.66025404e-01],
       [-5.00000000e-01,  8.66025404e-01,  6.44070374e-18],
       [-7.50000000e-01, -4.33012702e-01,  5.00000000e-01]])

In [406]:

rx = ground.expressVectorInAnotherFrame(state, osim.Vec3(*r[:,0]), J1_frame).to_numpy()
# rx = r[:, 0]
np.cross(rx, (I_in_joint - O_in_joint)/np.linalg.norm(I_in_joint - O_in_joint))

array([-2.32113328e-16,  4.48682456e-01, -7.44643624e-01])

In [364]:
model.getCoordinateSet().get(0).getAngularVelocity()

AttributeError: 'Coordinate' object has no attribute 'getAngularVelocity'

In [363]:
pl = (J[0, 3]*e1*ni) + (J[1, 3]*e2*ni) + (J[2, 3]*e3*ni) 
print(pl)

# print(blade_type(pl))
pos, d = g3c_tools.line_to_point_and_direction(L)
print(d)
n = to_vector(J[:3, 3])
print(d - (d|n)*n)
print(abs(d - (d|n)*n))

(0.43301^e14) + (0.43301^e15) - (0.5^e24) - (0.5^e25) - (0.75^e34) - (0.75^e35)
-(0.49416^e1) - (0.74464^e2) - (0.44868^e3)
-(0.70844^e1) - (0.49721^e2) - (0.07754^e3)
0.8689738957861244


In [326]:
rotationAxis: osim.Coordinate = osim.Coordinate.safeDownCast(joint1.getCoordinate(0))
axis = np.array([0, 0, 0] + J[3:, 0].tolist())
print(axis)
n = to_vector(J[3:, 0])
n = to_vector([1,0,0])
print(n)
Lp = L - (L | n)*n
# Lp = (L | n) / n
print(Lp)
print(abs(Lp))
# a = fd

[ 0.         0.         0.         0.4330127 -0.5       -0.75     ]
(1^e1)
(1.15458^e234) + (1.15458^e235) - (0.74464^e245) - (0.44868^e345)
0.869373379322482


array([[-2.91,  3.49,  0.45,  0.43,  0.25,  0.87],
       [ 0.83,  0.48, -5.71, -0.5 ,  0.87,  0.  ],
       [-2.23,  2.97, -0.77, -0.75, -0.43,  0.5 ],
       [ 0.43,  0.25,  0.87,  0.  ,  0.  ,  0.  ],
       [-0.5 ,  0.87,  0.  ,  0.  ,  0.  ,  0.  ],
       [-0.75, -0.43,  0.5 ,  0.  ,  0.  ,  0.  ]])

In [296]:
g3c_tools.random_line() / g3c_tools.random_line()

-0.09738 - (0.95397^e12) - (0.12779^e13) + (1.97233^e14) + (1.97233^e15) - (0.25324^e23) - (8.58553^e24) - (8.58553^e25) - (2.71368^e34) - (2.71368^e35) - (10.18885^e1234) - (10.18885^e1235)

In [233]:
(((O_in_joint_cf - I_in_joint_cf) | Px) / Px).normal()

(0.9172^e2) + (0.39843^e3)

In [212]:
rx = L | (Px^no)
ry = L | (Py^no)
rz = L | (Pz^no)

print(rx, ry, rz)
np.array([rx, ry, rz]).astype(float)
(J[3:, :3]) @ np.array([rx, ry, rz]).astype(float)

1.45903 0.61976 -0.12243


array([ 0.68069082, -0.19278995, -1.42385074])

In [151]:
w1_in_joint = model.getGround().expressVectorInAnotherFrame(state, osim.Vec3(*J[3:, 0]), J1_frame).to_numpy()
print(w1_in_joint.round(4))

w2_in_joint = model.getGround().expressVectorInAnotherFrame(state, osim.Vec3(*J[3:, 1]), J1_frame).to_numpy()
print(w2_in_joint.round(4))

w3_in_joint = model.getGround().expressVectorInAnotherFrame(state, osim.Vec3(*J[3:, 2]), J1_frame).to_numpy()
print(w3_in_joint.round(4))

w4_in_joint = model.getGround().expressVectorInAnotherFrame(state, osim.Vec3(*J[:3, 3]), J1_frame).to_numpy()
print(w4_in_joint.round(4))

w5_in_joint = model.getGround().expressVectorInAnotherFrame(state, osim.Vec3(*J[:3, 4]), J1_frame).to_numpy()
print(w5_in_joint.round(4))

w6_in_joint = model.getGround().expressVectorInAnotherFrame(state, osim.Vec3(*J[:3, 5]), J1_frame).to_numpy()
print(w6_in_joint.round(4))

[ 1.  0. -0.]
[ 0.  1. -0.]
[-0. -0.  1.]
[ 1.  0. -0.]
[ 0.  1. -0.]
[-0. -0.  1.]


In [154]:
L1 = J[:,0]
L1[:3] = 0
print(L1)
L2 = J[:,1]
L2[:3] = 0
print(L2)

[ 0.         0.         0.         0.4330127 -0.5       -0.75     ]
[ 0.         0.         0.         0.25       0.8660254 -0.4330127]


In [39]:
no^ni

-(1.0^e45)

In [264]:
# POSTURE 2
state2 = osim.State(state)
print(state2)
for c in model.getCoordinateSet():
    c.setValue(state2, np.random.uniform(-np.pi, np.pi))
model.equilibrateMuscles(state2)

lever_arms = np.array([muscle.computeMomentArm(state2, c) for c in model.getCoordinateSet()])
print(lever_arms)

J = common.station_jacobian(model, state2, osim.Vec3(4,0,3), bodyC)
Jw = common.frame_jacobian(model, state2, bodyC)[:3,:]

j = joint1.getParentFrame()
# j = bodyC
O_in_ground = bodyA.findStationLocationInGround(state2, origin).to_numpy()
I_in_ground = bodyC.findStationLocationInGround(state2, insertion).to_numpy()

O_in_j1 = bodyA.findStationLocationInAnotherFrame(state2, origin, joint1.getParentFrame()).to_numpy()
I_in_j1 = bodyC.findStationLocationInAnotherFrame(state2, insertion, joint1.getParentFrame()).to_numpy()

O_in_j2 = bodyA.findStationLocationInAnotherFrame(state2, origin, joint2.getParentFrame()).to_numpy()
I_in_j2 = bodyC.findStationLocationInAnotherFrame(state2, insertion, joint2.getParentFrame()).to_numpy()


# print(O_in_ground)
# print(I_in_ground)

u_in_ground = I_in_ground - O_in_ground
u1 = I_in_j1 - O_in_j1
u2 = I_in_j2 - O_in_j2

print()
print(lever_arms)
ground: osim.Ground = model.getGround()

t2 = np.random.uniform(600,1200) 
W2 = np.vstack([ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, i]), joint1.getParentFrame()).to_numpy() for i in [0,1,2]])
g2 = t2 * W2 @ (np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1))
print(g2)
print(g2/t2)
print()
print(ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, 0]), joint1.getParentFrame()).to_numpy().dot(np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1)))
print(ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, 1]), joint1.getParentFrame()).to_numpy().dot(np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1)))
print(ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, 2]), joint1.getParentFrame()).to_numpy().dot(np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1)))
# print()
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 3]), joint2.getParentFrame()).to_numpy().dot(np.cross(u2, O_in_j2) / np.linalg.norm(u2)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 4]), joint2.getParentFrame()).to_numpy().dot(np.cross(u2, O_in_j2) / np.linalg.norm(u2)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*Jw[:, 5]), joint2.getParentFrame()).to_numpy().dot(np.cross(u2, O_in_j2) / np.linalg.norm(u2)))

# If translations
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 3]), joint2.getParentFrame()).to_numpy().dot(- u2 / np.linalg.norm(u2)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 4]), joint2.getParentFrame()).to_numpy().dot(- u2 / np.linalg.norm(u2)))
# print(ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 5]), joint2.getParentFrame()).to_numpy().dot(- u2 / np.linalg.norm(u2)))

print(ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, 1]), joint1.getParentFrame()))
WW2 = ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, 1]), joint1.getParentFrame()).to_numpy()
WW2 = WW2 @ np.array([e1, e2, e3])
ww2 = WW2
WW2 = WW2 ^ ni

WW1 = ground.expressVectorInAnotherFrame(state2, osim.Vec3(*Jw[:, 0]), joint1.getParentFrame()).to_numpy()
WW1 = WW1 @ np.array([e1, e2, e3])
ww1 = WW1
WW1 = WW1 ^ ni
# print(WW2)
# print(I_in_j1)
Pdual = (np.cross(I_in_j1, O_in_j1) / np.linalg.norm(u1)) @ np.array([e1, e2, e3])
normal = Pdual
Ot = O_in_j1 @ np.array([e1, e2, e3])
It = I_in_j1 @ np.array([e1, e2, e3])
print(Pdual)
fw = (g2/t2)[1]
print((WW2 | Pdual) | (no^ni).inv())
print((WW2 | Pdual).dual())
print("h", fw, "j", (fw*ni))

print((g2/t2))
G2 = g2/t2
alpha = G2[1]/G2[0]
# print(alpha*G2[0], G2[1])
equa1_lh = -G2[0]^ni
equa1_rh = WW1 | normal
equa2_lh = -G2[1]^ni
equa2_rh = WW2 | normal

equa2_adapted_lh = -(G2[1]/alpha)^ni
equa2_adapted_rh = (WW2/alpha) | normal
# Now equa1 lh = equa2 lh
print(equa1_rh - equa2_adapted_rh)

LG = ((WW1-WW2/alpha))
print(LG)
print(normal.dual())

print(LG ^ normal.dual())
print(normal.dual())
# print(LG)
# print(WW2.join(WW1))
print(ww1^ww2^ni)
# print(G2[1]/alpha)
# print(equa1_lh / equa2_lh)
# print(-fw^ni)
# print(((WW3) | (Ot^It^ni)))

# normal = 
# WW1 = e1 + e2 + e3

<opensim.simbody.State; proxy of <Swig Object of type 'SimTK::State *' at 0x000002C5286714D0> >
[ 0.65727173 -0.26644266  0.16396265 -0.70016109]

[ 0.65727173 -0.26644266  0.16396265 -0.70016109]
[ 700.87443647 -284.11818727  174.83976241]
[ 0.65727173 -0.26644266  0.16396265]

0.6572717306264522
-0.266442664945883
0.16396265470854365
~[0,1,0]
(0.65727^e1) - (0.26644^e2) + (0.16396^e3)
-(0.26644^e4) - (0.26644^e5)
(0.26644^e1234) + (0.26644^e1235)
h -0.266442664945883 j -(0.26644^e4) - (0.26644^e5)
[ 0.65727173 -0.26644266  0.16396265]
0
(1.0^e14) + (1.0^e15) + (2.46684^e24) + (2.46684^e25)
-(0.16396^e1245) - (0.26644^e1345) - (0.65727^e2345)
0
-(0.16396^e1245) - (0.26644^e1345) - (0.65727^e2345)
(1.0^e124) + (1.0^e125)
0


In [126]:
M1 = np.linalg.inv(W1) @ (g1)
M2 = np.linalg.inv(W2) @ (g2)

P1 = ho.VectorSpace(M1)
P2 = ho.VectorSpace(M2)
print(P1.orthogonal_projection_matrix @ bodyC.findStationLocationInAnotherFrame(state, insertion, joint1.getParentFrame()).to_numpy())
print(P2.orthogonal_projection_matrix @ bodyC.findStationLocationInAnotherFrame(state2, insertion, joint1.getParentFrame()).to_numpy())

zero1_in_C = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(0), bodyC).to_numpy()
zero2_in_C = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(0),bodyC).to_numpy()

zero1_in_A = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(0), bodyA).to_numpy()
zero2_in_A = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(0), bodyA).to_numpy()

M1_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*M1), bodyC).to_numpy()
M2_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(*M2),bodyC).to_numpy()
# print(M1_I, M2_I)

M1_O = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*M1), bodyA).to_numpy()
M2_O = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(*M2), bodyA).to_numpy()
# print(M1_O, M2_O)


# # P1_I = ho.Line(M1_I - zero1_in_C)
# # P1_I = ho.Plane(P1_I.generators, zero1_in_C)

# P2_I = ~ho.VectorSpace(M2_I - zero2_in_C)
# P2_I = ho.Plane(P2_I.generators, zero2_in_C)

# L = P1_I & P2_I
# print("gen", L.generators.ravel(), L.location.ravel())
# # print(L.generators.ravel() / np.linalg.norm(L.generators.ravel()))
# k = insertion.to_numpy() - zero1_in_C
# k = k /np.linalg.norm(k)
# print(k)
# # L_orth = ~L
# # test = L_orth.orthogonal_projection_matrix @ (insertion.to_numpy() - L.location.ravel())
# # print("test", test)



T1 = ho.AffineSpace(M1_I - zero1_in_C, zero1_in_C)
T2 = ho.AffineSpace(M2_I - zero2_in_C, zero2_in_C)
print("h", (T1 & T2).location)
# point_of_common = (T1 & T2).location.ravel()
print(((~T1) & (~T2)).location)
P_inter = (~T1) & (~T2)
print("g", P_inter.generators.ravel() - P_inter.location.ravel())
print("g", insertion.to_numpy() - P_inter.location.ravel())

LI = np.cross(M1_I, M2)
LI = LI / np.linalg.norm(LI) #we are in joint1 child frame
print(LI)

LI_ = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*LI), bodyC).to_numpy()
print(LI_ - zero1_in_C)
d = LI_ - zero1_in_C
d = d / np.linalg.norm(d)
p = zero1_in_C
# print(zero1_in_C)


# LO = np.cross(M1_O, M2_O)
# LI = LI / np.linalg.norm(LI)
# LO = LO / np.linalg.norm(LO)
# print(LI, LO)
print(insertion.to_numpy(), origin.to_numpy())

[ 0.00000000e+00 -2.77555756e-17  6.93889390e-18]
[ 2.22044605e-16  5.55111512e-17 -6.93889390e-18]
h [-0.96946277  1.10768818 -0.26441816]
[ 0.28314    -0.10534595 -0.13891745]
g [-0.72765675 -0.753431   -0.11585107]
g [0.41686    0.80534595 0.23891745]
[-0.03365256 -0.06785634 -0.99712738]
[ 0.23032412 -0.02731577 -0.97273051]
[0.7 0.7 0.1] [0.9 0.8 0.8]


In [128]:
print(M1_I - zero1_in_C)

# A = M1_I - zero1_in_C
#  = M2_I - zero2_in_C
I = insertion.to_numpy() @ np.array([e1, e2, e3])

# T1 = up(zero1_in_C @ np.array([e1,e2,e3])) ^ ((M1_I - zero1_in_C) @ np.array([e1, e2, e3])) ^ ni
# # T1 = no ^ (M1_I @ np.array([e1, e2, e3])) ^ ni
# T2 = (M2_I - zero2_in_C) @ np.array([e1, e2, e3])
# print((I - (zero1_in_C @ np.array([e1,e2,e3]))) ^ T1)
# # T1 = up(zero1_in_C @ np.array([e1,e2,e3])) ^ T1 ^ ni
# # T2 = up(zero2_in_C @ np.array([e1,e2,e3])) ^ T2 ^ ni

# # K = T1.dual().meet(T2.dual())
# # K
# # I^T2
# # .meet(T2.dual())

M1 = np.linalg.inv(W1) @ (g1)
M2 = np.linalg.inv(W2) @ (g2)

P1 = ~ho.VectorSpace(M1)
P2 = ~ho.VectorSpace(M2)

p1, p2, p3 = P1.orthogonal_projection_matrix @ np.random.uniform(size=3), P1.orthogonal_projection_matrix @ np.random.uniform(size=3), P1.orthogonal_projection_matrix @ np.random.uniform(size=3)

q1, q2, q3 = P2.orthogonal_projection_matrix @ np.random.uniform(size=3), P2.orthogonal_projection_matrix @ np.random.uniform(size=3), P2.orthogonal_projection_matrix @ np.random.uniform(size=3)

M1_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*M1), bodyC).to_numpy()
zero1_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(0), bodyC).to_numpy()
p1_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*p1), bodyC).to_numpy()
p2_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*p2), bodyC).to_numpy()
p3_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state, osim.Vec3(*p3), bodyC).to_numpy()
q1_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(*q1), bodyC).to_numpy()
q2_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(*q2), bodyC).to_numpy()
q3_I = joint1.getParentFrame().findStationLocationInAnotherFrame(state2, osim.Vec3(*q3), bodyC).to_numpy()

p1 = up(p1_I @ np.array([e1, e2, e3]))
p2 = up(p2_I @ np.array([e1, e2, e3]))
p3 = up(p3_I @ np.array([e1, e2, e3]))

q1 = up(q1_I @ np.array([e1, e2, e3]))
q2 = up(q2_I @ np.array([e1, e2, e3]))
q3 = up(q3_I @ np.array([e1, e2, e3]))

u = (M1_I @ np.array([e1,e2,e3])) - (zero1_I @ np.array([e1,e2,e3]))
dual_plane1 = (u + ((zero1_I @ np.array([e1,e2,e3])) | u)*ni)
plane1 = dual_plane1.dual()
# plane1 = p1^p2^p3^ni
plane2 = q1^q2^q3^ni

# print(I^plane1)
# print(I^plane2)
inter = plane1.meet(plane2)
print(up(I)^inter)
print(up(I)^plane1)
print(up(I)^plane2)

print(inter)

[-432.30224739  223.99476799   -0.76914901]


c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


0
0
0
(18.10365^e124) + (18.10365^e125) + (8.35855^e134) + (8.35855^e135) + (27.75129^e145) + (5.77232^e234) + (5.77232^e235) + (53.61365^e245) + (15.90526^e345)


c:\Users\glaisne\mambaforge\envs\workspace\Lib\site-packages\numba\core\decorators.py:262: NumbaDeprecationWarning: numba.generated_jit is deprecated. Please see the documentation at: https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-generated-jit for more information and advice on a suitable replacement.
  warnings.warn(msg, NumbaDeprecationWarning)


In [41]:
ground.expressVectorInAnotherFrame(state, osim.Vec3(*J[:, 3]), joint2.getParentFrame()).to_numpy()

array([ 1.00000000e+00,  2.77555756e-17, -1.11022302e-16])

In [158]:

matter: osim.SimbodyMatterSubsystem = model.getMatterSubsystem()
Qdot = state.getQDot()
print(Qdot)

~[0 0 0 0]


In [268]:
wm = np.zeros((3,3))
wm[2,:] = 1
np.linalg.pinv(wm)

array([[0.        , 0.        , 0.33333333],
       [0.        , 0.        , 0.33333333],
       [0.        , 0.        , 0.33333333]])